In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq
from datetime import date 
from pathlib import Path
import shutil
import stat

In [ ]:
# VARIAVEIS E FUNCOES

# caminho dos arquivos csv's de movimentação
path_movimetacao = Path("C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/arquivos_nubank/conta")

# caminho dos arquivos csv's de gastos em créditos
path_credito = Path("C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/arquivos_nubank/cartao_credito")

# data atual para ser utilizada como parte do nome das pastas em que será salvo os arquivos parquet
today = date.today().isoformat()

# caminho para salvar arquivo parquet final na pasta bronze
path_bronze = Path(f"C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/data/bronze/{today}")

# Remove a pasta e arquivos dentro dela caso já exista (path_bronze)
def remove_readonly(func, path, _):
    """Altera permissão para escrita e tenta remover o arquivo novamente"""
    os.chmod(path, stat.S_IWRITE)
    func(path)

NameError: name 'Path' is not defined

In [ ]:
# Excluir a pasta se ela já existir
if os.path.exists(path_bronze):
    shutil.rmtree(path_bronze, onerror=remove_readonly)

# criar o caminho para salvar os arquivos
os.makedirs(path_bronze)

In [3]:
## Função para unificar os arquivos e retornar em um dataframe para que possa salvar na camada raw

def unifica_arquivos(local_arquivo):
    lista_dfs = []
        
    for arquivo in os.listdir(local_arquivo):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(local_arquivo, arquivo)
            df = pd.read_csv(caminho_arquivo)
            lista_dfs.append(df)

    arquivo_final = pd.concat(lista_dfs, ignore_index = True)

    return arquivo_final

In [4]:
# ingestão dos dados dos csv's de movimentação da conta

df_movimentacao = unifica_arquivos(path_movimetacao)

caminho_bronze_movimentacao = Path(path_bronze/"movimentacao.parquet")

df_movimentacao.to_parquet(caminho_bronze_movimentacao, engine = "pyarrow", index = False)

In [5]:
# ingestão dos dados dos csv's de compras em crédito

df_credito = unifica_arquivos(path_credito)

caminho_bronze_credito = Path(path_bronze/"credito.parquet")

df_credito.to_parquet(caminho_bronze_credito, engine = "pyarrow", index = False)